In [1]:
from actual_work.preprocessing import Preprocessing
from actual_work.classifier import Classifier

prep = Preprocessing("offenseval-training-v1.tsv")
"""
[1] SVMClassifier
[2] RandomForest_Classifier
[3] LogisticalRegression_Classifier
[4] NaiveBayesClassifier
[5] KNNClassifier
[6] DTClassifier
"""
clf = Classifier(prep.train_tweets, prep.train_labels, prep.test_tweets, prep.test_labels, 1)

<Figure size 640x480 with 1 Axes>

Optimal C: 0.17
0.7398791540785499
